In [2]:
import os
import json
from glob import glob
import random
# from tqdm import tqdm
# tqdm for notebooks

from tqdm import tqdm_notebook as tqdm
import pandas as pd
from fuzzywuzzy import fuzz
from moviepy.editor import VideoFileClip

/run/determined/pythonuserbase/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# create folder for each dataset first    

In [3]:
def save_json(content, save_path):
    # if no such directory, create one
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_json(filename):
    with open(filename, "r") as f:
        return json.load(f)

# qvh

In [5]:
ann_root = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/QVHighlights'
# ann_root = "../../data/annotations"
train_path = ann_root + '/highlight_train_release.jsonl'
val_path = ann_root + '/highlight_val_release.jsonl'
test_path = ann_root + '/highlight_test_release.jsonl'

train_mcq_path = ann_root + '/Meta-Llama-3.1-70B-Instruct_qv_highlights_multiple_choice_qa.json'

In [6]:
train = load_jsonl(train_path)
val = load_jsonl(val_path)
test = load_jsonl(test_path)

train_mcq = load_json(train_mcq_path)

In [ ]:
def process_QVH(data, relative_time=False, save_float=False, is_test=False):
    out = []
    for d in data:
        sample = {}
        sample['video'] = d['vid']
        sample['qid'] = 'QVHighlight_' + str(d['qid'])
        sample['query'] = d['query']
        duration = d['duration']
        sample['duration'] = duration

        if not is_test:
            windows = d['relevant_windows']
            if relative_time:
                relative_time_windows = []
                for window in windows:
                    start = window[0] / duration
                    end = window[1] / duration

                    if save_float:
                        relative_time_windows.append([round(start, 2), round(end, 2)])
                    else:
                        relative_time_windows.append([int(round(start, 2) * 100), int(round(end, 2) * 100)])
                sample['relevant_windows'] = relative_time_windows
            else:
                sample['relevant_windows'] = windows
        else:
            sample['relevant_windows'] = [[0, 150]] # dummy value

        out.append(sample)

    return out

In [ ]:
save_float = False
relative_time = False

new_train = process_QVH(train, relative_time=relative_time, save_float=save_float)
new_val = process_QVH(val, relative_time=relative_time, save_float=save_float)
new_test = process_QVH(test, relative_time=relative_time, save_float=save_float, is_test=True)

In [ ]:
# save data
if save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative_float.json')
    save_json(new_val, ann_root + '/lavis/val_relative_float.json')
    save_json(new_test, ann_root + '/lavis/test_relative_float_dummy.json')
elif save_float and not relative_time:
    save_json(new_train, ann_root + '/lavis/train_float.json')
    save_json(new_val, ann_root + '/lavis/val_float.json')
    save_json(new_test, ann_root + '/lavis/test_float_dummy.json')
elif not save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative.json')
    save_json(new_val, ann_root + '/lavis/val_relative.json')
    save_json(new_test, ann_root + '/lavis/test_relative_dummy.json')
else:
    save_json(new_train, ann_root + '/lavis/train.json')
    save_json(new_val, ann_root + '/lavis/val.json')
    save_json(new_test, ann_root + '/lavis/test_dummy.json')

In [57]:
# process QVH_QA

errors_no_similar_answer = []

def process_QVH_QA(data_mc, data_og, relative_time=False, save_float=False):
    out = []
    for d in data_og:
        sample = {}
        sample['video'] = d['vid']
        sample['qid'] = 'QVHighlight_' + str(d['qid'])
        duration = d['duration']
        sample['duration'] = duration
        windows = d['relevant_windows']

        # get data from multiple choice
        if str(d['qid']) in data_mc:
            d_q = data_mc[str(d['qid'])]
        else:
            # print('Error: qid {} not in data_mc'.format(d['qid']))
            continue

        sample['query'] = d_q['question']
        sample['question'] = d_q['question']
        sample['answer_str'] = d_q['answer']
        options = d_q['choices']
        # split options by comma
        options = options.split(',')
        # remove leading and trailing whitespaces
        options = [x.strip() for x in options]
        
        sample['num_option'] = len(options)
        
        for i, option in enumerate(options):
            sample[f"a{i}"] = option

        # get correct answer index
        try:
            correct_idx = options.index(d_q['answer'])
        except:
            # if the correct answer does not 100% match any of the options, look for the most similar one
            closest_answer = None
            closest_score = 0
            for i, option in enumerate(options):
                score = fuzz.ratio(d_q['answer'], option)
                if score > closest_score:
                    closest_score = score
                    closest_answer = i
            
            errors_no_similar_answer.append(d['qid'])
            if closest_score > 40:
                correct_idx = closest_answer
            else:
                print('Error: no similar answer found')
                print('Error: >{}< not in {}'.format(d_q['answer'], options))
                continue

        sample['answer'] = correct_idx

        if relative_time:
            relative_time_windows = []
            for window in windows:
                start = window[0] / duration
                end = window[1] / duration

                if save_float:
                    relative_time_windows.append([round(start, 2), round(end, 2)])
                else:
                    relative_time_windows.append([int(round(start, 2) * 100), int(round(end, 2) * 100)])
            sample['relevant_windows'] = relative_time_windows
        else:
            sample['relevant_windows'] = windows

        out.append(sample)

    return out

In [ ]:
save_float = False
relative_time = False

new_train_mc = process_QVH_QA(train_mcq, train, relative_time=relative_time, save_float=save_float)

In [52]:
# save data
if not save_float and not relative_time:
    save_json(new_train_mc, ann_root + '/lavis/train_mcqa.json')

# Charades

In [ ]:
# read csv from /pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/Charades/Charades_v1_train.csv
train_df = pd.read_csv('/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/Charades/Charades_original/Charades_v1_train.csv', delimiter=',')
test_df = pd.read_csv('/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/Charades/Charades_original/Charades_v1_test.csv', delimiter=',')

In [ ]:
# get all unique ids
train_ids = train_df["id"].unique()
print(len(train_ids))

# randomly select 800 ids for validation
random.seed(42)
random.shuffle(train_ids)

val_ids = train_ids[:800]
train_ids = train_ids[800:]

len(val_ids), len(train_ids)

In [ ]:
ann_root = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/Charades/Charades_STA'
train_path = ann_root + '/train.txt'
test_path = ann_root + '/test.txt'

In [ ]:
def process_charades_STA(data_path, df, video_ids=None, relative_time=False, save_float=False):
    # read txt and put each line into new element in list
    with open(data_path) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    out = []

    for s in content:
        # format "id start end##query"
        s = s.split('##') # -> [id start end, query]
        query = s[1] # -> query
        s = s[0] # -> id start end
        s = s.split(' ') # -> [[id], [start], [end]]
        id = s[0] # -> id

        if video_ids is not None and id not in video_ids:
            continue
        
        # get meta data from df using id
        # get row with id == id
        row = df.loc[df["id"] == id]
        values = row.values[0]

        # get duration
        duration = values[10]

        # convert to float
        s[1] = float(s[1])
        s[2] = float(s[2])
        if s[2] > duration:
            s[2] = duration

        if relative_time:
            # convert to relative time
            s[1] = s[1] / duration
            s[2] = s[2] / duration

            if save_float:
                # For float conversion
                window = [round(s[1], 2), round(s[2], 2)] # -> [start, end]
                assert window[0] >= 0 and window[1] <= 1
            else:
                # For int conversion -> round to nearest int
                window = [int(s[1] * 100), int(s[2] * 100)]
                assert window[0] >= 0 and window[1] <= 100
        else:
            if save_float:
                # For float conversion
                window = [float(s[1]), float(s[2])] # -> [start, end]
            else:
                # For int conversion -> round to nearest int
                window = [round(float(s[1])), round(float(s[2]))]

        # get objects
        objects = values[7]
        # only split if objects is not nan or contains ; (which means multiple objects)
        try:
            objects = objects.split(';')
        except:
            print('no objects: ', objects, ' for id: ', id)
            objects = []

        out.append(
            {
                'id': id,
                'query': query,
                'window': [window],
                'duration': duration,
                'objects': objects
            }
        )

    return out

In [ ]:
save_float = True
relative_time = False

train = process_charades_STA(train_path, train_df, train_ids, relative_time=relative_time, save_float=save_float)
val = process_charades_STA(train_path, train_df, val_ids, save_float=save_float)
test = process_charades_STA(test_path, test_df, save_float=save_float)

new_train = []
new_val = []
new_test = []
for i, qa in enumerate(train):
    qa_dict = {}
    qa_dict['video'] = qa['id']
    qa_dict['qid'] = 'Charades-STA_' + str(qa['id'])
    qa_dict['query'] = qa['query']
    qa_dict['duration'] = qa['duration']
    qa_dict['relevant_windows'] = qa['window']
    qa_dict['objects'] = qa['objects']
    new_train.append(qa_dict)

for i, qa in enumerate(val):
    qa_dict = {}
    qa_dict['video'] = qa['id']
    qa_dict['qid'] = 'Charades-STA_' + str(qa['id'])
    qa_dict['query'] = qa['query']
    qa_dict['duration'] = qa['duration']
    qa_dict['relevant_windows'] = qa['window']
    qa_dict['objects'] = qa['objects']
    new_val.append(qa_dict)

for i, qa in enumerate(test):
    qa_dict = {}
    qa_dict['video'] = qa['id']
    qa_dict['qid'] = 'Charades-STA_' + str(qa['id'])
    qa_dict['query'] = qa['query']
    qa_dict['duration'] = qa['duration']
    qa_dict['relevant_windows'] = qa['window']
    qa_dict['objects'] = qa['objects']
    new_test.append(qa_dict)

len(new_train), len(new_val), len(new_test)

In [ ]:
# save data
if save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/new_train_relative_float.json')
    save_json(new_val, ann_root + '/lavis/new_val_relative_float.json')
    save_json(new_test, ann_root + '/lavis/test_relative_float.json')
elif save_float and not relative_time:
    save_json(new_train, ann_root + '/lavis/new_train_float.json')
    save_json(new_val, ann_root + '/lavis/new_val_float.json')
    save_json(new_test, ann_root + '/lavis/test_float.json')
elif not save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/new_train_relative.json')
    save_json(new_val, ann_root + '/lavis/new_val_relative.json')
    save_json(new_test, ann_root + '/lavis/test_relative.json')
else:
    save_json(new_train, ann_root + '/lavis/new_train.json')
    save_json(new_val, ann_root + '/lavis/new_val.json')
    save_json(new_test, ann_root + '/lavis/test.json')

In [ ]:
# For processing without the custom data slipt, i.e. having only the original train and test split

save_float = True
relative_time = False

train = process_charades_STA(train_path, train_df, None, relative_time=relative_time, save_float=save_float)
test = process_charades_STA(test_path, test_df, save_float=save_float)

new_train = []
new_val = []
new_test = []
for i, qa in enumerate(train):
    qa_dict = {}
    qa_dict['video'] = qa['id']
    qa_dict['qid'] = 'Charades-STA_' + str(qa['id'])
    qa_dict['query'] = qa['query']
    qa_dict['duration'] = qa['duration']
    qa_dict['relevant_windows'] = qa['window']
    qa_dict['objects'] = qa['objects']
    new_train.append(qa_dict)

for i, qa in enumerate(test):
    qa_dict = {}
    qa_dict['video'] = qa['id']
    qa_dict['qid'] = 'Charades-STA_' + str(qa['id'])
    qa_dict['query'] = qa['query']
    qa_dict['duration'] = qa['duration']
    qa_dict['relevant_windows'] = qa['window']
    qa_dict['objects'] = qa['objects']
    new_test.append(qa_dict)

len(new_train), len(new_test)

In [ ]:
# save data
if save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative_float.json')
    save_json(new_test, ann_root + '/lavis/test_relative_float.json')
elif save_float and not relative_time:
    save_json(new_train, ann_root + '/lavis/train_float.json')
    save_json(new_test, ann_root + '/lavis/test_float.json')
elif not save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative.json')
    save_json(new_test, ann_root + '/lavis/test_relative.json')
else:
    save_json(new_train, ann_root + '/lavis/train.json')
    save_json(new_test, ann_root + '/lavis/test.json')

# NextQA

In [3]:
ann_root = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/NExT_QA'
raw_root = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/raw/NExT'
train_path = ann_root + '/nextqa/train.csv'
val_path = ann_root + '/nextqa/val.csv'
test_path = ann_root + '/nextqa/test.csv'
map_vid_vidorID_path = ann_root + '/map_vid_vidorID.json'

In [4]:
raw_train = pd.read_csv(train_path, delimiter=',')
raw_val = pd.read_csv(val_path, delimiter=',')
train = []
val = []
key = ['video', 'question', 'a0', 'a1', 'a2', 'a3', 'a4', 'answer', 'qid', 'type'] 
for i in range(len(raw_train)):
    data = {}
    for k in key:
        data[k] = raw_train.iloc[i][k]
    train.append(data)

for i in range(len(raw_val)):
    data = {}
    for k in key:
        data[k] = raw_val.iloc[i][k]
    val.append(data) 

In [5]:
vid_map = load_json(map_vid_vidorID_path)

In [9]:
def get_video_duration(vid):
    vid_path = raw_root + "/" + vid_map[vid] + '.mp4'
    clip = VideoFileClip(vid_path)
    return clip.duration

In [10]:
new_train = []
new_val = []

print('Processing train and val data...')
print('This could lake a while (100 min), because we need to extract the video durations for each video')

for qa in train:
    qa_dict = {}
    qa_dict['video'] = vid_map[str(qa['video'])]
    qa_dict['duration'] = get_video_duration(str(qa['video']))
    qa_dict['num_option'] = int(5)
    qa_dict['qid'] = '_'.join([qa['type'], str(qa['video']), str(qa['qid'])])
    for i in range(5):
        qa_dict['a{}'.format(str(i))] = qa['a{}'.format(str(i))]+'.'
    qa_dict['answer'] = int(qa['answer'])
    qa_dict['question'] = qa['question']+'?'
    new_train.append(qa_dict)

for qa in val:
    qa_dict = {}
    qa_dict['video'] = vid_map[str(qa['video'])]
    qa_dict['duration'] = get_video_duration(str(qa['video']))
    qa_dict['num_option'] = int(5)
    qa_dict['qid'] = '_'.join([qa['type'], str(qa['video']), str(qa['qid'])])
    for i in range(5):
        qa_dict['a{}'.format(str(i))] = qa['a{}'.format(str(i))]+'.'
    qa_dict['answer'] = int(qa['answer'])
    qa_dict['question'] = qa['question']+'?'
    new_val.append(qa_dict)

In [11]:
save_json(new_train, ann_root + '/lavis/train.json')
save_json(new_val, ann_root + '/lavis/val.json')

# NExT-GQA

In [4]:
ann_root = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/NExT_QA'
raw_root = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/raw/NExT'
# train_path = ann_root + '/nextgqa/train.csv'
val_path = ann_root + '/nextgqa/val.csv'
test_path = ann_root + '/nextgqa/test.csv'
map_vid_vidorID_path = ann_root + '/map_vid_vidorID.json'

In [5]:
raw_val = pd.read_csv(val_path, delimiter=',')
raw_test = pd.read_csv(test_path, delimiter=',')
val = []
test = []
key = ['video_id', 'question', 'a0', 'a1', 'a2', 'a3', 'a4', 'answer', 'qid', 'type'] 

for i in range(len(raw_val)):
    data = {}
    for k in key:
        data[k] = raw_val.iloc[i][k]
    val.append(data)

for i in range(len(raw_test)):
    data = {}
    for k in key:
        data[k] = raw_test.iloc[i][k]
    test.append(data)

In [6]:
vid_map = load_json(map_vid_vidorID_path)

In [7]:
time_val = load_json(ann_root + '/nextgqa/gsub_val.json')
time_test = load_json(ann_root + '/nextgqa/gsub_test.json')

In [8]:
def get_video_duration(vid):
    vid_path = raw_root + "/" + vid_map[vid] + '.mp4'
    clip = VideoFileClip(vid_path)
    return clip.duration

def get_answer_idx(answer, options):
    for i, option in enumerate(options):
        if option == answer:
            return i
    print('Error: answer not in options')
    return -1

In [9]:
def get_moment_timespan(time_data, video_id, qid):
    data = time_data[str(video_id)]
    location = data['location'][str(qid)]
    duration = data['duration']

    return location, duration

In [10]:
new_val = []
new_test = []

# print('Processing train and val data...')
# print('This could lake a while (100 min), because we need to extract the video durations for each video')

for qa in val:
    qa_dict = {}
    qa_dict['video'] = vid_map[str(qa['video_id'])]
    # qa_dict['duration'] = get_video_duration(str(qa['video_id']))
    qa_dict['num_option'] = int(5)
    qa_dict['qid'] = '_'.join([qa['type'], str(qa['video_id']), str(qa['qid'])])
    for i in range(5):
        qa_dict['a{}'.format(str(i))] = qa['a{}'.format(str(i))]+'.'
    qa_dict['answer'] = get_answer_idx(qa['answer'], [qa['a0'], qa['a1'], qa['a2'], qa['a3'], qa['a4']])
    qa_dict['question'] = qa['question']+'?'

    ### GQA specific
    relevant_windows, duration = get_moment_timespan(time_val, qa['video_id'], qa['qid'])
    qa_dict['relevant_windows'] = relevant_windows
    qa_dict['duration'] = duration

    new_val.append(qa_dict)

for qa in test:
    qa_dict = {}
    qa_dict['video'] = vid_map[str(qa['video_id'])]
    # qa_dict['duration'] = get_video_duration(str(qa['video_id']))
    qa_dict['num_option'] = int(5)
    qa_dict['qid'] = '_'.join([qa['type'], str(qa['video_id']), str(qa['qid'])])
    for i in range(5):
        qa_dict['a{}'.format(str(i))] = qa['a{}'.format(str(i))]+'.'
    qa_dict['answer'] = get_answer_idx(qa['answer'], [qa['a0'], qa['a1'], qa['a2'], qa['a3'], qa['a4']])
    qa_dict['question'] = qa['question']+'?'

    ### GQA specific
    relevant_windows, duration = get_moment_timespan(time_test, qa['video_id'], qa['qid'])
    qa_dict['relevant_windows'] = relevant_windows
    qa_dict['duration'] = duration

    new_test.append(qa_dict)

In [11]:
save_json(new_val, ann_root + '/lavis/nextgqa/val.json')
save_json(new_test, ann_root + '/lavis/nextgqa/test.json')

# Epic Sounds

Do the same as above for the other datasets but now also ectract save the audio files (.wav) for the relevant moments in the dataset.

In [3]:
path_to_annotations = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/EpicSounds/epic-sounds-annotations'
train_path = path_to_annotations + '/EPIC_Sounds_train.csv'
val_path = path_to_annotations + '/EPIC_Sounds_validation.csv'
test_path = path_to_annotations + '/EPIC_Sounds_recognition_test_timestamps.csv'

train_df = pd.read_csv(train_path, delimiter=',')
val_df = pd.read_csv(val_path, delimiter=',')
test_df = pd.read_csv(test_path, delimiter=',')

path_to_raw = '/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/raw/EPIC-KITCHENS'

In [ ]:
train_df.head()

In [12]:
def extract_audio_segment(video, output_path, start_time, end_time):
    if start_time > end_time:
        print('Start time is greater than end time')
        return

    def safe_subclip(video, start, end):
        return video.audio.subclip(start, min(end, video.duration))

    # Extract the audio from the specified time range
    # audio_clip = video.audio.subclip(start_time, end_time, verbose=False)
    audio_clip = safe_subclip(video, start_time, end_time)
    
    # Write the audio to a file
    # if file does not already exist
    # if not os.path.exists(output_path):
    audio_clip.write_audiofile(output_path, verbose=False)

In [6]:
def timestamp_to_seconds(timestamp):
    parts = timestamp.split(':')
    
    if len(parts) == 3:  # Format: HH:MM:SS.mmm
        hours, minutes, seconds = parts
    elif len(parts) == 2:  # Format: MM:SS.mmm
        hours = '0'
        minutes, seconds = parts
    else:
        raise ValueError("Invalid timestamp format")
    
    seconds, milliseconds = seconds.split('.')
    
    total_seconds = (
        int(hours) * 3600 +
        int(minutes) * 60 +
        int(seconds) +
        int(milliseconds) / 1000
    )
    
    return total_seconds

In [15]:
def proccess_EPIC_Sounds(df, split="train" ,relative_time=False, save_float=False):
    assert split in ["train", "val", "test"]

    out = []

    video_cache = {}

    for i, row in df.iterrows():
        sample = {}
        sample['video'] = row['video_id']
        sample['qid'] = 'EPIC-Sounds_' + str(row['annotation_id'])

        video_path = os.path.join(path_to_raw, row['participant_id'], "videos", row['video_id'] + '.MP4')

        ### get the video and its duration if not already in cache #############################################
        if row['video_id'] not in video_cache:
            if len(video_cache) > 10:
                print("Freeing up cache:", video_cache.keys())
                for video in video_cache.values():
                    video['VideoFileClip'].close()
                video_cache = {}

            print(f'loading video: {video_path}')
            video = VideoFileClip(video_path, verbose=False)
            print(f'duration: {video.duration}')
            video_cache[row['video_id']] = {'VideoFileClip': video, 'duration': video.duration}
        
        sample['duration'] = video_cache[row['video_id']]['duration']

        ########################################################################################################

        ### get audio sequence from video given start and end time for audio ###################################
        audio_path = os.path.join(path_to_raw, 'EpicSounds', split, str(row['annotation_id']) + '.wav')
        start = row['start_timestamp']
        end = row['stop_timestamp']
        # convert time format (00:00:02.466 or 01:56.091) to float
        start = timestamp_to_seconds(start)
        end = timestamp_to_seconds(end)
        
        # extract audio
        extract_audio_segment(video_cache[row['video_id']]['VideoFileClip'], audio_path, start, end)
        
        sample['query'] = audio_path
        ########################################################################################################

        # if the class key exists
        if 'class' in row:
            sample['class'] = row['class']

        if relative_time:
            if save_float:
                sample['relevant_windows'] = [[round(start, 2), round(end, 2)]]
            else:
                sample['relevant_windows'] = [[int(round(start, 2) * 100), int(round(end, 2) * 100)]]
        else:
            if save_float:
                sample['relevant_windows'] = [[round(start, 2), round(end, 2)]]
            else:
                sample['relevant_windows'] = [[round(start), round(end)]]

        out.append(sample)
    
    # clear cache
    for video in video_cache.values():
        video['VideoFileClip'].close()

    return out

In [ ]:
save_float = True
relative_time = False

# new_train = proccess_EPIC_Sounds(train_df, split="train", relative_time=relative_time, save_float=save_float)
new_val = proccess_EPIC_Sounds(val_df, split="val", save_float=True)
# new_test = proccess_EPIC_Sounds(test_df, split="test", save_float=True)

In [ ]:
# save data
if save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative_float.json')
    save_json(new_test, ann_root + '/lavis/test_relative_float.json')
elif save_float and not relative_time:
    save_json(new_train, ann_root + '/lavis/train_float.json')
    save_json(new_test, ann_root + '/lavis/test_float.json')
elif not save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative.json')
    save_json(new_test, ann_root + '/lavis/test_relative.json')
else:
    save_json(new_train, ann_root + '/lavis/train.json')
    save_json(new_test, ann_root + '/lavis/test.json')

# TACoS

In [ ]:
import os
from glob import glob
# import VideoFileClip
from moviepy.editor import VideoFileClip

In [ ]:
base_path = "/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/raw/"
directory_avi = os.path.join(base_path, "videos")
directory_mp4 = os.path.join(base_path, "TACoS")

In [ ]:
# convert avi to mp4 

def convert_avi_to_mp4(filename, avi_path, target_path):
    os.system('ffmpeg -i {} {}'.format(
        os.path.join(avi_path, filename),
        os.path.join(target_path, filename.replace('.avi', '.mp4'))
    ))

# get all avi files
avi_files = glob(os.path.join(directory_avi, "*.avi"))
avi_files = [os.path.basename(f) for f in avi_files]

# convert all avi files to mp4
for avi_file in avi_files:
    # if file already exists, skip
    if not os.path.exists(os.path.join(directory_mp4, avi_file.replace('.avi', '.mp4'))):
        convert_avi_to_mp4(avi_file, directory_avi, directory_mp4)

In [ ]:
# downscale resolution to 224x224
target_path = os.path.join(directory_mp4, "res_224")

def downscale_resolution(filename, source_path, target_path):
    os.system('ffmpeg -i {} -vf scale=224:224 {}'.format(
        os.path.join(source_path, filename),
        os.path.join(target_path, filename)
    ))

# get all mp4 files
mp4_files = glob(os.path.join(directory_mp4, "*.mp4"))
mp4_files = [os.path.basename(f) for f in mp4_files]

# downscale all mp4 files to 224x224
for mp4_file in mp4_files:
    # if file already exists, skip
    if not os.path.exists(os.path.join(target_path, mp4_file)):
        downscale_resolution(mp4_file, directory_mp4, target_path)

In [ ]:
ann_root = "/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/TACoS"
train_path = os.path.join(ann_root, "train.jsonl")
val_path = os.path.join(ann_root, "val.jsonl")
test_path = os.path.join(ann_root, "test.jsonl")


video_path = "/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/raw/TACoS"

In [ ]:
# read jsonl
train = load_jsonl(train_path)
val = load_jsonl(val_path)
test = load_jsonl(test_path)

# show length of each split
len(train), len(val), len(test)

In [ ]:
save_float = False
relative_time = True

new_train = []
new_val = []
new_test = []
for i, qa in enumerate(train):
    qa_dict = {}
    qa_dict['video'] = qa['vid']
    qa_dict['qid'] = 'TACoS_' + str(qa['qid'])
    qa_dict['query'] = qa['query']
    # round duration to 2 decimal places
    qa_dict['duration'] = round(qa['duration'], 2)
    
    assert len(qa['relevant_windows']) == 1

    start, end = qa['relevant_windows'][0]
    if relative_time:
        # convert to relative time
        start = start / qa['duration']
        end = end / qa['duration']

        if save_float:
            # For float conversion
            window = [round(start, 2), round(end, 2)] # -> [start, end]
            assert window[0] >= 0 and window[1] <= 1
            qa_dict['relevant_windows'] = [window]
        else:
            # For int conversion -> round to nearest int
            window = [int(round(start, 2) * 100), int(round(end, 2) * 100)]
            assert window[0] >= 0 and window[1] <= 100
            qa_dict['relevant_windows'] = [window]
    else:
        if save_float:
            # For float conversion
            window = [float(start), float(end)] # -> [start, end]
            qa_dict['relevant_windows'] = [window]
        else:
            # For int conversion -> round to nearest int
            window = [round(float(start)), round(float(end))]
            qa_dict['relevant_windows'] = [window]
    
    # qa_dict['objects'] = qa['objects']
    new_train.append(qa_dict)

for i, qa in enumerate(val):
    qa_dict = {}
    qa_dict['video'] = qa['vid']
    qa_dict['qid'] = 'TACoS_' + str(qa['qid'])
    qa_dict['query'] = qa['query']
    # round duration to 2 decimal places
    qa_dict['duration'] = round(qa['duration'], 2)
    
    # round relevant windows to 2 decimal places
    assert len(qa['relevant_windows']) == 1
    if save_float:
        qa_dict['relevant_windows'] = [[float(round(x, 2)) for x in qa['relevant_windows'][0]]]
    else:
        qa_dict['relevant_windows'] = [[int(round(x)) for x in qa['relevant_windows'][0]]]
    
    
    # qa_dict['objects'] = qa['objects']
    new_val.append(qa_dict)

for i, qa in enumerate(test):
    qa_dict = {}
    qa_dict['video'] = qa['vid']
    qa_dict['qid'] = 'TACoS_' + str(qa['qid'])
    qa_dict['query'] = qa['query']
    # round duration to 2 decimal places
    qa_dict['duration'] = round(qa['duration'], 2)
    
    # round relevant windows to 2 decimal places
    assert len(qa['relevant_windows']) == 1
    if save_float:
        qa_dict['relevant_windows'] = [[float(round(x, 2)) for x in qa['relevant_windows'][0]]]
    else:
        qa_dict['relevant_windows'] = [[int(round(x)) for x in qa['relevant_windows'][0]]]
    
    # qa_dict['objects'] = qa['objects']
    new_test.append(qa_dict)

len(new_train), len(new_val), len(new_test)

In [ ]:
# save data
if save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative_float.json')
    save_json(new_val, ann_root + '/lavis/val_relative_float.json')
    save_json(new_test, ann_root + '/lavis/test_relative_float.json')
elif save_float and not relative_time:
    save_json(new_train, ann_root + '/lavis/train_float.json')
    save_json(new_val, ann_root + '/lavis/val_float.json')
    save_json(new_test, ann_root + '/lavis/test_float.json')
elif not save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative.json')
    save_json(new_val, ann_root + '/lavis/val_relative.json')
    save_json(new_test, ann_root + '/lavis/test_relative.json')
else:
    save_json(new_train, ann_root + '/lavis/train.json')
    save_json(new_val, ann_root + '/lavis/val.json')
    save_json(new_test, ann_root + '/lavis/test.json')

# ActivityNet Captions

In [ ]:
ann_root = "/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/ActivityNet"
train_path = os.path.join(ann_root, "train.json")
val_path = os.path.join(ann_root, "val_1.json")
test_path = os.path.join(ann_root, "val_2.json")


In [ ]:
train = load_json(train_path)
val = load_json(val_path)
test = load_json(test_path)

In [ ]:
def process_activitynet(data, relative_time=False, save_float=False):
    out = []

    for video_id, sample in data.items():
        duration = sample['duration']
        sentences = sample['sentences']
        timestamps = sample['timestamps']
        for j, (start, end) in enumerate(timestamps):

            if relative_time:
                # convert to relative time
                start = start / duration
                end = end / duration

                if save_float:
                    # For float conversion
                    window = [round(start, 2), round(end, 2)]
                    assert window[0] >= 0 and window[1] <= 1
                else:
                    # For int conversion -> round to nearest int
                    window = [int(round(start, 2) * 100), int(round(end, 2) * 100)]
                    assert window[0] >= 0 and window[1] <= 100
            else:
                if save_float:
                    # For float conversion
                    window = [float(start), float(end)]
                else:
                    # For int conversion -> round to nearest int
                    window = [round(float(start)), round(float(end))]

            new_sample = {
                'video': video_id,
                'qid': f'ActivityNet_{video_id}_{j}',
                'query': sentences[j],
                'duration': duration,
                'relevant_windows': [window]
            }

            out.append(new_sample)
    
    return out

In [ ]:
save_float = False
relative_time = False

new_train = process_activitynet(train, relative_time=relative_time, save_float=save_float)
new_val = process_activitynet(val, save_float=save_float)
new_test = process_activitynet(test, save_float=save_float)

len(new_train), len(new_val), len(new_test)

In [ ]:
# save data
if save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative_float.json')
    save_json(new_val, ann_root + '/lavis/val_relative_float.json')
    save_json(new_test, ann_root + '/lavis/test_relative_float.json')
elif save_float and not relative_time:
    save_json(new_train, ann_root + '/lavis/train_float.json')
    save_json(new_val, ann_root + '/lavis/val_float.json')
    save_json(new_test, ann_root + '/lavis/test_float.json')
elif not save_float and relative_time:
    save_json(new_train, ann_root + '/lavis/train_relative.json')
    save_json(new_val, ann_root + '/lavis/val_relative.json')
    save_json(new_test, ann_root + '/lavis/test_relative.json')
else:
    save_json(new_train, ann_root + '/lavis/train.json')
    save_json(new_val, ann_root + '/lavis/val.json')
    save_json(new_test, ann_root + '/lavis/test.json')

# ANet TAL

In [ ]:
ann_root = "/pfss/mlde/workspaces/mlde_wsp_Rohrbach/data/annotations/ActivityNet_TAL"
data = load_json(ann_root + '/anet_tal.json')

In [ ]:
templates_all = [
    'a photo of {}.',
    'a photo of a person {}.',
    'a photo of a person using {}.',
    'a photo of a person doing {}.',
    'a photo of a person during {}.',
    'a photo of a person performing {}.',
    'a photo of a person practicing {}.',
    'a video of {}.',
    'a video of a person {}.',
    'a video of a person using {}.',
    'a video of a person doing {}.',
    'a video of a person during {}.',
    'a video of a person performing {}.',
    'a video of a person practicing {}.',
    'a example of {}.',
    'a example of a person {}.',
    'a example of a person using {}.',
    'a example of a person doing {}.',
    'a example of a person during {}.',
    'a example of a person performing {}.',
    'a example of a person practicing {}.',
    'a demonstration of {}.',
    'a demonstration of a person {}.',
    'a demonstration of a person using {}.',
    'a demonstration of a person doing {}.',
    'a demonstration of a person during {}.',
    'a demonstration of a person performing {}.',
    'a demonstration of a person practicing {}.',
]

templates_video = [
    'a video of {}.',
    'a video of a person {}.',
    'a video of a person using {}.',
    'a video of a person doing {}.',
    'a video of a person during {}.',
    'a video of a person performing {}.',
    'a video of a person practicing {}.',
    'a example of {}.',
    'a example of a person {}.',
    'a example of a person using {}.',
    'a example of a person doing {}.',
    'a example of a person during {}.',
    'a example of a person performing {}.',
    'a example of a person practicing {}.',
    'a demonstration of {}.',
    'a demonstration of a person {}.',
    'a demonstration of a person using {}.',
    'a demonstration of a person doing {}.',
    'a demonstration of a person during {}.',
    'a demonstration of a person performing {}.',
    'a demonstration of a person practicing {}.',
]

In [ ]:
def process_activitynet_tal(data, relative_time=False, save_float=False):
    out_train = []
    out_val = []
    out_test = []

    for video_id, sample in data.items():
        video_id = "v_" + video_id
        duration = sample['duration']
        split = sample['subset']
        annotations = sample['annotations']
        windows = []
        for ann in annotations:
            start = ann['segment'][0]
            end = ann['segment'][1]

            # label will always be the same for all annotations/ windows
            label = ann['label']

            if relative_time:
                # convert to relative time
                start = start / duration
                end = end / duration

                if save_float:
                    # For float conversion
                    window = [round(start, 2), round(end, 2)]
                    assert window[0] >= 0 and window[1] <= 1
                else:
                    # For int conversion -> round to nearest int
                    window = [int(round(start, 2) * 100), int(round(end, 2) * 100)]
                    assert window[0] >= 0 and window[1] <= 100
            else:
                if save_float:
                    # For float conversion
                    window = [float(start), float(end)]
                else:
                    # For int conversion -> round to nearest int
                    window = [round(float(start)), round(float(end))]
            
            window.extend([label])

            windows.append(window)

        # get one example of the template and insert the label in lower case
        # query = random.choice(templates_video).format(label.lower())
        query = ""

        new_sample = {
            'video': video_id,
            'qid': f'ActivityNet_{video_id}_0',
            'query': query,
            'duration': duration,
            'relevant_windows': windows
        }

        if split == 'training':
            out_train.append(new_sample)
        elif split == 'validation':
            out_val.append(new_sample)
        elif split == 'testing':
            out_test.append(new_sample)
    
    return out_train, out_val, out_test

In [ ]:
save_float = False
relative_time = False

train, val, test = process_activitynet_tal(data["database"], relative_time=relative_time, save_float=save_float)

len(train), len(val), len(test)

In [ ]:
# save data
if save_float and relative_time:
    save_json(train, ann_root + '/lavis/train_relative_float.json')
    save_json(val, ann_root + '/lavis/val_relative_float.json')
    save_json(test, ann_root + '/lavis/test_relative_float.json')
elif save_float and not relative_time:
    save_json(train, ann_root + '/lavis/train_float.json')
    save_json(val, ann_root + '/lavis/val_float.json')
    save_json(test, ann_root + '/lavis/test_float.json')
elif not save_float and relative_time:
    save_json(train, ann_root + '/lavis/train_relative.json')
    save_json(val, ann_root + '/lavis/val_relative.json')
    save_json(test, ann_root + '/lavis/test_relative.json')
else:
    save_json(train, ann_root + '/lavis/train.json')
    save_json(val, ann_root + '/lavis/val.json')
    save_json(test, ann_root + '/lavis/test.json')

In [ ]:
# get all unique classes from activitynet
classes = []
for video_id, sample in data["database"].items():
    annotations = sample['annotations']
    for ann in annotations:
        label = ann['label']
        if label not in classes:
            classes.append(label)
len(classes)

In [ ]:
# save to txt file
with open('classes.txt', 'w') as f:
    for item in classes:
        f.write("%s\n" % item)